<a href="https://colab.research.google.com/github/angelrecalde2024/Power-System-Planning-and-Transmission-Design-2026/blob/main/OptimizationSimple_MultiperiodTransportLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# pip install pyomo highspy
from pyomo.environ import ConcreteModel, Set, Param, Var, NonNegativeReals, Objective, Constraint, summation, minimize
from pyomo.contrib.appsi.solvers.highs import Highs  # HiGHS via APPSI (uses the 'highspy' wheel)

# -----------------------------
# Tiny multi-period transport LP
# Plants i ∈ I ship to markets j ∈ J across periods t ∈ T
# Minimize cost  Σ_t Σ_i Σ_j  c[i,j]*x[i,j,t]
# s.t.
#   For each (i,t):  Σ_j x[i,j,t] ≤ supply[i]               (capacity per period)
#   For each (j,t):  Σ_i x[i,j,t] ≥ demand[j,t]             (meet demand)
# -----------------------------

I_plants = ['P1', 'P2']
J_markets = ['M1', 'M2', 'M3']
T_periods = [1, 2, 3]  # think hourly or daily; scales easily

cost = {('P1','M1'): 2, ('P1','M2'): 4, ('P1','M3'): 5,
        ('P2','M1'): 3, ('P2','M2'): 1, ('P2','M3'): 7}

supply = {'P1': 60, 'P2': 50}  # per period capacity
demand = {
    (1,'M1'): 20, (1,'M2'): 25, (1,'M3'): 10,
    (2,'M1'): 22, (2,'M2'): 18, (2,'M3'): 12,
    (3,'M1'): 15, (3,'M2'): 25, (3,'M3'): 20,
}

m = ConcreteModel()

# Sets
m.I = Set(initialize=I_plants)
m.J = Set(initialize=J_markets)
m.T = Set(initialize=T_periods)

# Params
m.c = Param(m.I, m.J, initialize=lambda m,i,j: cost[i,j], within=NonNegativeReals)
m.s = Param(m.I, initialize=lambda m,i: supply[i], within=NonNegativeReals)
m.d = Param(m.T, m.J, initialize=lambda m,t,j: demand[t,j], within=NonNegativeReals)

# Decision vars: shipment from plant i to market j at period t
m.x = Var(m.I, m.J, m.T, within=NonNegativeReals)

# Objective
def total_cost(m):
    return sum(m.c[i,j]*m.x[i,j,t] for i in m.I for j in m.J for t in m.T)
m.OBJ = Objective(rule=total_cost, sense=minimize)

# Capacity constraints (auto-expands to |I|×|T| constraints)
def cap_rule(m, i, t):
    return sum(m.x[i,j,t] for j in m.J) <= m.s[i]
m.Capacity = Constraint(m.I, m.T, rule=cap_rule)

# Demand constraints (auto-expands to |J|×|T| constraints)
def dem_rule(m, t, j):
    return sum(m.x[i,j,t] for i in m.I) >= m.d[t,j]
m.Demand = Constraint(m.T, m.J, rule=dem_rule)

# Solve with HiGHS (APPSI interface)
solver = Highs()
res = solver.solve(m)
print(res.termination_condition)

# Print a few optimal shipments
for t in m.T:
    print(f"\nPeriod {t}")
    for j in m.J:
        shipped = {i: round(m.x[i,j,t](), 2) for i in m.I if m.x[i,j,t]() > 1e-7}
        print(f"  Market {j}: {shipped}")
print("\nOptimal cost:", round(m.OBJ(), 2))


TerminationCondition.optimal

Period 1
  Market M1: {'P1': 20.0}
  Market M2: {'P2': 25.0}
  Market M3: {'P1': 10.0}

Period 2
  Market M1: {'P1': 22.0}
  Market M2: {'P2': 18.0}
  Market M3: {'P1': 12.0}

Period 3
  Market M1: {'P1': 15.0}
  Market M2: {'P2': 25.0}
  Market M3: {'P1': 20.0}

Optimal cost: 392.0
